In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [2]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize Text

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [14]:
model = Sequential()
model.add(Dense(150, input_shape=(max_sequence_len-1,), activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 150)               2400      
                                                                 
 dense_13 (Dense)            (None, 150)               22650     
                                                                 
 dense_14 (Dense)            (None, 150)               22650     
                                                                 
 dense_15 (Dense)            (None, 930)               140430    
                                                                 
Total params: 188130 (734.88 KB)
Trainable params: 188130 (734.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=150, verbose=1)

Epoch 1/150
71/71 [==============================] - 1s 2ms/step - loss: 0.9012 - accuracy: 0.7289
Epoch 2/150
71/71 [==============================] - 0s 2ms/step - loss: 0.8572 - accuracy: 0.7484
Epoch 3/150
71/71 [==============================] - 0s 2ms/step - loss: 0.8444 - accuracy: 0.7471
Epoch 4/150
71/71 [==============================] - 0s 2ms/step - loss: 0.8757 - accuracy: 0.7284
Epoch 5/150
71/71 [==============================] - 0s 2ms/step - loss: 0.8542 - accuracy: 0.7387
Epoch 6/150
71/71 [==============================] - 0s 2ms/step - loss: 0.8600 - accuracy: 0.7391
Epoch 7/150
71/71 [==============================] - 0s 2ms/step - loss: 0.9327 - accuracy: 0.7351
Epoch 8/150
71/71 [==============================] - 0s 2ms/step - loss: 1.0006 - accuracy: 0.7173
Epoch 9/150
71/71 [==============================] - 0s 2ms/step - loss: 1.0517 - accuracy: 0.7080
Epoch 10/150
71/71 [==============================] - 0s 2ms/step - loss: 1.0244 - accuracy: 0.7236
Epoch 11/

## Save Model for JS

In [20]:

output_dir = "./tfjs_models/FFNN1" 
tfjs.converters.save_keras_model(model, output_dir)

## Save word index

In [11]:
wordIndex = {}
for word in tokenizer.word_index:
    wordIndex[word] = tokenizer.word_index[word]

with open('./tokenizer/wordIndex.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [12]:
with open('./tokenizer/max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))